In [1]:
import pandas as pd
import numpy as np
from scipy import stats
import matplotlib.pyplot as plt
import datetime

In [11]:
# переделал на подсчет зон окончания поездки dropoff_longitude,dropoff_latitude
# функция по подготовке файла к нужному формату
def F_total(file_name):
    data = pd.read_csv(file_name, sep=',', header=0)
    data.columns = [k.strip() for k in data.columns] # убираем пробелы из заголовков столбцов   

    # в 2016 году 'tpep_pickup_datetime' и 'tpep_dropoff_datetime' в 2014 без tpep_
    #Фильтруем от поездок с: нулевой'tpep_pickup_datetime' длительностью
    
    data = data[(data['tpep_pickup_datetime'] != data['tpep_dropoff_datetime'])] # !!!поменять для 2016 года tpep_
    #нулевым количеством пассажиров
    data = data[data['passenger_count'] != 0]
    #нулевым расстоянием поездки по счётчику
    data = data[data['trip_distance'] != 0]
    # Нью-Йорк вписан в прямоугольник от -74.25559 до -73.70001 градусов долготы и от 40.49612 до 40.91553 широты.
    data = data[(data['dropoff_longitude'] >= -74.25559) & (data['dropoff_longitude'] <= -73.70001)]
    data = data[(data['dropoff_latitude'] >= 40.49612) & (data['dropoff_latitude'] <= 40.91553)]
    # Агрегируйте данные за май 2016 года по часам и областям
    # добавим столбец только с датой и часами    
    data['hours'] = data.tpep_pickup_datetime.str[:13]    # !!!поменять для 2016 года tpep_
    
    # делим нью-йорк на 2500 квадратов
    # Нью-Йорк вписан в прямоугольник от -74.25559 до -73.70001 градусов долготы и от 40.49612 до 40.91553 широты.
    dolg = np.linspace(-74.25559,-73.70001,51)
    shir = np.linspace(40.49612,40.91553,51)  
    
    # считаем количество вызовов по ячейкам
#    NY_cub = stats.binned_statistic_2d(data.pickup_longitude.values, data.pickup_latitude.values,  
#                                       None, 'count', range=[[-74.25559,-73.70001],[40.49612,40.91553]], 
#                                       bins=[dolg,shir])
    # считаем количество высадок по ячейкам

    NY_cub = stats.binned_statistic_2d(data.dropoff_longitude.values, data.dropoff_latitude.values,  
                                       None, 'count', range=[[-74.25559,-73.70001],[40.49612,40.91553]], 
                                       bins=[dolg,shir])
    
    # массив с регионами
    reg_arr = np.zeros(len(NY_cub.binnumber), dtype=int)
    data['regions'] = 0
    
    #переводим регионы к нашим, т.к. функция делает квадрат 52х52, x-(x/52*2+50)
    #ппц на это убил день чтобы понять =(

    for i in range(len(NY_cub.binnumber)):
        reg_arr[i] = NY_cub.binnumber[i] - (NY_cub.binnumber[i]/52*2+50)    
    
    data.regions = reg_arr
    
    # добавим поле со счетчиком регионов, так удобнее вызывать pivot_table
    data['count_r'] = 1
    
    # делаем группировку  по часам и регионам
    dt_gr = data.pivot_table(values=['count_r'],index=['hours','regions'], aggfunc='count', fill_value = 0)    

    # делаем группировку по Total_amount, Passenger_count, Trip_distance, Fare_amount
    dt_feature = data.pivot_table(values=['total_amount', 'passenger_count', 'trip_distance', 'fare_amount']
                                  ,index=['hours','regions'], aggfunc='sum', fill_value = 0) 
    
    # выведем часы из индекса в обычный столбец
    dt_gr.reset_index(inplace=True) 
    
    # дополнительно сделаем группировку только по часам - всего в мае 31*24 = 744 часа 
    dt_hours = data.pivot_table(values=['regions'],index=['hours'], aggfunc='count', fill_value = 0)  
    # выведем часы из индекса в обычный столбец 
    dt_hours.reset_index(inplace=True)   

    # добавляем 0 в регионы без заказов, получается массив для мая 744*2500 = 1 860 000 
    arr_dt_gr = dt_gr.values  # массив по ненулевым заказам
    arr_dt_hours = dt_hours.values # массив с группировкой по всем часам

    arr_null = [] # массив квадратов с 0 заказами
    k = 0 # счетчик в массиве arr_dt_gr
    for i in range(len(arr_dt_hours)): #len(arr_dt_hours)
        for j in range(2500):
            if (arr_dt_gr[k][0] == arr_dt_hours[i][0]): #часы совпадают
                if (int(arr_dt_gr[k][1]) == j+1):              #квадрат не пустой
                    arr_null.append([arr_dt_gr[k][0], int(arr_dt_gr[k][1]), int(arr_dt_gr[k][2])])
                    if (k < len(arr_dt_gr)-1): k+=1
                else:                                   #квадрат пустой
                    arr_null.append([arr_dt_gr[k][0], j+1, 0])
            else:                                       # часы не совпадают
                arr_null.append([arr_dt_hours[i][0], j+1, 0])

    # датафрейм с развернутый по часам и ячейкам
    dt_itog = pd.DataFrame(arr_null) 
    dt_itog.columns=['data','region','count']
    
    # сгруппируем часам и регионам 744х2500
    dt_itog_kvadrat = dt_itog.pivot_table(values=['count'],index=['data'], columns=['region'], 
                                          aggfunc='sum', fill_value = 0)
    
    dt_itog_kvadrat.reset_index(inplace=True) # выведем часы из индекса в обычный столбец
    dt_itog_kvadrat.columns = [k for j,k in dt_itog_kvadrat.columns] # переименовываем столбцы    
    dt_itog_kvadrat.rename(columns={'': 'date_hour'}, inplace=True)

    dt_123 = dt_itog_kvadrat.drop([dt_itog_kvadrat.columns[0]], axis=1)
    dt_123 = dt_123*1.0
    dt_123.insert(loc=0, column='date_hour', value=dt_itog_kvadrat.date_hour.values)
    
#    dt_123.to_csv('kv_'+file_name,sep=';') # пишем в файл труды нашей группировки
#    dt_feature.to_csv('feat_'+file_name,sep=';') # пишем в файл для новых фич
    dt_gr.to_csv('droff_'+file_name,sep=';') # пишем в файл для зон высадки
    return dt_gr

In [17]:
%%time
file_m = 'yellow_tripdata_2016-06.csv'
c = F_total(file_m)

Wall time: 1min 4s


In [10]:
print c.shape
c.head(2)

(281327, 3)


,hours,regions,count_r
0,2016-01-01 00,-50,1
1,2016-01-01 00,-23,1
